## Semantic Router Dynamic Routing

Allows you to dynamically set variables to use in functions

In [1]:
import os

from getpass import getpass
from dotenv import load_dotenv
from datetime import datetime
from zoneinfo import ZoneInfo
from semantic_router import Route
from semantic_router.layer import RouteLayer
from semantic_router.encoders import OpenAIEncoder
from semantic_router.utils.function_call import get_schema

In [2]:
load_dotenv()

True

### Set up static routes

In [3]:
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

In [5]:
encoder = OpenAIEncoder()
rl = RouteLayer(encoder=encoder, routes=routes)

2024-01-20 10:56:22 INFO semantic_router.utils.logger Initializing RouteLayer


In [6]:
rl("how's the weather today?")

RouteChoice(name='chitchat', function_call=None, similarity_score=None, trigger=None)

### Set up dynamic routes: Must be in sphinx

In [7]:
def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London". Do NOT put the place
        name itself like "rome", or "new york", you must provide
        the IANA format.
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")

In [8]:
get_time("America/New_York")

'12:56'

In [9]:
# get the function schema
schema = get_schema(get_time)
schema

{'name': 'get_time',
 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
 'signature': '(timezone: str) -> str',
 'output': "<class 'str'>"}

In [10]:
time_route = Route(
    name="get_time",
    utterances=[
        "what is the time in new york city?",
        "what is the time in london?",
        "I live in Rome, what time is it?",
    ],
    function_schema=schema,
)

In [11]:
rl.add(time_route)

2024-01-20 10:56:23 INFO semantic_router.utils.logger Adding `get_time` route


### Router Example

What we're doing is setting up the ageantic workflow where the agent will decide what to do and generate the inputs. We're removing the decision layer and using semanics to generate the logic.

In [31]:
out = rl("what is the time in America/New_York?") # works
#out = rl("what is the time in new york city?") # does not work
#out = rl("what is the time in New_York?") # does not work
#out = rl("what is the time in New York?") # does not work

2024-01-20 11:29:50 INFO semantic_router.utils.logger Extracting function input...


In [32]:
out

RouteChoice(name='get_time', function_call={'timezone': 'America/New_York'}, similarity_score=None, trigger=None)

In [33]:
get_time(**out.function_call)

'13:29'